## Annotate fasta file with the Uniprot Swissprot database

If you do not have the Uniprot/Swissprot Database, download and format it:

In [ ]:
wget ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
gunzip uniprot_sprot.dat.gz
makeblastdb -in uniprot_sprot.pep -dbtype prot

Perform the BLAST search

In [ ]:
blastp -query {Query_File.fasta} -db prot -outfmt 6 -out Blast_Results.txt

Append results to the original fasta file (called "Query_File.fasta" in this example:

In [ ]:
# this code requires GNU sed (gsed); if you do not have it install with homebrew
brew install gsed
# Extract the relevent lines from the BLAST output
cut -f 1,2 Blast_Results.txt > tmp0

###########################################################################
# convert fasta file into a document with each sequence on a single line #
###########################################################################
# add a tab to the end of each sequence ID
gsed s/"\(>\)\(.*\)"/"\1\2\t"/g Query_File.fasta > tmp1 
# convert to a single line
tr -d '\n' < tmp1  > tmp2
# replace '>' with a newline
gsed 's/>/\n/g' tmp2 > tmp3
# remove first blank line (necessary for join command)
gsed -r '/^\s*$/d' tmp3 > tmp4

# join BLAST results with query sequences
join <(sort tmp0) <(sort tmp4) > tmp5

# reformat into a fasta file
sort -u tmp5 > tmp6
gsed 's/^/>/g' tmp6 > tmp7
gsed -E 's/ ([^ ]*)$/\n\1/' tmp7 > tmp8
gsed -E 's/^([^ ]*) /\1|/' tmp8 > Query_File.Annotated.fasta
# cleanup
rm tmp*

Take a careful look at the results. With this code, any sequence that lacks a hit in the Swissprot Uniprot databse will be removed.

## Annotate fasta file with the PFAM database


If you do not have the pfamA Database, download and format it:

In [ ]:
wget
gunzip Pfam-A.hmm.gz
hmmpress Pfam-A.hmm

Search for protein domains with HMMscan. If you install `Pfam-A.hmm` in a different directory from the search, you will need to include the path.

In [ ]:
hmmscan  --domtblout HMMER.out \
--pfamtblout HMMER.pfam.txt Pfam-A.hmm \
Query_File.fasta > HMMER.pfam.log 

Extract hmm information to create list for samtools

In [ ]:
###########################
# reformat HMMER.out file #
###########################
# convert all spaces into single tabs
tr -s '[:blank:]' '\t' < HMMER.out > tmp1
# remove comment lines (lines that start with '#')
sed '/^#/d' tmp1 > tmp2

# extract lines with conserved domain of interest (in this example, 'Lycopene_cycl')
sed -n '/^Lycopene_cycl/p' tmp2 > tmp3

# extract complete seqeunces
sort -u <(cut -f 4 tmp3) > tmp4
# OR extract the sequence and 'ali' coordinates using awk and the ASCII octal digits for " (042)
awk '{print "\042"$4"\042"":"$18"-"$19}' tmp3 > tmp4

# extract results through samtools
xargs samtools faidx Query_File.fasta < tmp4 > HMMER_Results.fasta
# cleanup
rm tmp*